In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

c:\anaconda3\envs\fastcampus_ml\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
# 데이터 불러오기
data = pd.read_csv('./data/kc_house_data.csv')
data.head()

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [3]:
'''
id: 집 고유아이디
date: 집이 팔린 날짜
price: 집 가격(타겟변수)
bedrooms: 주택당 침실개수
bathrooms: 주택당 화장실 개수
floors: 전체 층 개수
waterfront: 해변이 보이는지 (0, 1)
condition: 집 청소상태 (1~5)
grade: King County Grading System으로 인한 평점 (1~13)
yr_built: 집이 지어진 년도
yt_renovated: 집이 리모델링된 년도
zipcode: 우편번호
lat: 위도
long: 경도
'''

'\nid: 집 고유아이디\ndate: 집이 팔린 날짜\nprice: 집 가격(타겟변수)\nbedrooms: 주택당 침실개수\nbathrooms: 주택당 화장실 개수\nfloors: 전체 층 개수\nwaterfront: 해변이 보이는지 (0, 1)\ncondition: 집 청소상태 (1~5)\ngrade: King County Grading System으로 인한 평점 (1~13)\nyr_built: 집이 지어진 년도\nyt_renovated: 집이 리모델링된 년도\nzipcode: 우편번호\nlat: 위도\nlong: 경도\n'

In [4]:
ncar = data.shape[0]
nvar = data.shape[1]

print(ncar)
print(nvar)

21613
14


### 무의미하게 판단되는 변수 제거

In [7]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis=1)

### 설명변수와 타겟변수를 분리, 학습/평가데이터 분리

In [17]:
feature_columns = list(data.columns.difference(['price']))

X = data[feature_columns]
y = data['price']

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=42) # 7:3 으로 학습/평가데이터 분리
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) #3 데이터 차원 확인

(15129, 8) (6484, 8) (15129,) (6484,)


### 학습데이터를 선형회귀모형 적합 후 평가데이터로 검증

In [18]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sm_train_x = sm.add_constant(train_x, has_constant='add')
sm_model = sm.OLS(train_y, sm_train_x)
fitted_sm_model = sm_model.fit()
fitted_sm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Sun, 18 Oct 2020   Prob (F-statistic):               0.00
Time:                        22:36:19   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
sm_test_x = sm.add_constant(test_x, has_constant='add')
sm_model_predict = fitted_sm_model.predict(sm_test_x)

In [24]:
sqrt(mean_squared_error(sm_model_predict, test_y)) # RMSE 값

239804.29670858168

### Bagging 구현을 통한 일반회귀모델과 성능비교

In [50]:
# for문으로 bagging 구현

import random

bagging_predict_result = [] 

for _ in range(30):
    data_index = [data_index for data_index in range(train_x.shape[0])]
    random_data_index = np.random.choice(data_index, train_x.shape[0])
    sm_train_x = train_x.iloc[random_data_index,]
    sm_train_y = train_y.iloc[random_data_index,]
    sm_train_x = sm.add_constant(sm_train_x, has_constant='add')
    sm_model = sm.OLS(sm_train_y, sm_train_x)
    fitted_sm_model = sm_model.fit()
    pred = fitted_sm_model.predict(sm_test_x)
    bagging_predict_result.append(pred)
    print(sqrt(mean_squared_error(pred, test_y)))

240129.8492463595
239694.60448511862
239948.1856794934
241043.96577074117
239716.2158658496
239719.11064467524
240594.3127601503
240168.1261413766
239994.7253154653
240464.3018077149
240219.35663294225
240516.0714700639
240320.10520395785
240036.87593573207
240111.9673038997
240474.81941085326
239619.90544941498
239709.2269973137
240438.55583696545
240225.65930203832
241933.8632231549
239899.90563165044
240459.8413171711
240752.9736705139
239783.74687789322
240460.63553922973
241169.21650226827
241191.5377236755
239878.20414935233
240223.41588130803


In [51]:
bagging_predict_result[0]

735      5.577809e+05
2830     7.131169e+05
4106     1.099635e+06
16218    1.456560e+06
19964    6.897132e+05
             ...     
12606    5.967845e+05
14393    6.809664e+05
6899     3.225181e+05
85       8.959050e+05
21363    4.302250e+05
Length: 6484, dtype: float64

In [52]:
bagging_predict = []

for lst2_index in range(test_x.shape[0]):
    temp_predict = []
    
    for lst_index in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[lst_index].values[lst2_index])
        
    bagging_predict.append(np.mean(temp_predict))

In [54]:
sqrt(mean_squared_error(bagging_predict, test_y))
# 이 경우는 Bagging 성능이 일반선형회귀 모델의 성능과 큰 차이가 없다

240029.03193371673

### 선형회귀모형에 적합 후 평가데이터 검증

In [76]:
from sklearn.linear_model import LinearRegression

regression_model = LinearRegression()

linear_model1 = regression_model.fit(train_x, train_y)
predict1= linear_model1.predict(test_x)

print(sqrt(mean_squared_error(predict1, test_y)))

239804.29670858147


### Bagging 을 이용하여 선형외귀모형에 적합 후 평가 (Sampling 5번)

In [85]:
from sklearn.ensemble import BaggingRegressor

bagging_model = BaggingRegressor(base_estimator=regression_model, n_estimators=5)
linear_model2 = bagging_model.fit(train_x, train_y)
predict2 = linear_model2.predict(test_x)

print(sqrt(mean_squared_error(predict2, test_y)))

239976.66779136762


### 샘플링을 많이하는 경우

In [86]:
from sklearn.ensemble import BaggingRegressor

bagging_model = BaggingRegressor(base_estimator=regression_model, n_estimators=30)
linear_model3 = bagging_model.fit(train_x, train_y)
predict3 = linear_model3.predict(test_x)

print(sqrt(mean_squared_error(predict3, test_y)))

239739.26886709


### 학습데이터를 의사결정나무모형에 적합 후 평가데이터 검증

In [89]:
from sklearn.tree import DecisionTreeRegressor

decision_tree_model = DecisionTreeRegressor()
tree_model = decision_tree_model.fit(train_x, train_y)
predict_tree = tree_model.predict(test_x)

print(sqrt(mean_squared_error(predict_tree, test_y)))

294614.46462270856


In [96]:
# for문으로 bagging 구현
import random
bagging_predict_result = [] 
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])]
    random_data_index = np.random.choice(data_index, train_x.shape[0])
    sm_train_x = train_x.iloc[random_data_index,]
    sm_train_y = train_y.iloc[random_data_index,]
    decision_tree_model = DecisionTreeRegressor()
    tree_model = decision_tree_model.fit(sm_train_x, sm_train_y)
    predict_tree = tree_model.predict(test_x)
    bagging_predict_result.append(predict_tree)
    print(sqrt(mean_squared_error(predict_tree, test_y)))

283162.7155177318
307122.77061849815
305018.3985194008
294119.9780585983
303144.89239704906
290049.22651720856
292085.65696765645
338127.9994640701
281161.9702999915
302374.30424628634


In [99]:
bagging_predict = []

for lst2_index in range(test_x.shape[0]):
    temp_predict = []
    
    for lst_index in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[lst_index][lst2_index])
        
    bagging_predict.append(np.mean(temp_predict))

In [101]:
sqrt(mean_squared_error(bagging_predict, test_y))
# 위의 10개 개별 Tree 모델의 성능보다 Bagging 모델의 성능이 높음

243689.7960536861

### Bagging 패키지 활용해서 Tree 모형 적합

In [126]:
from sklearn.ensemble import BaggingRegressor

# 5번 샘플링 
bagging_model = BaggingRegressor(base_estimator=decision_tree_model, n_estimators=5)
linear_model4 = bagging_model.fit(train_x, train_y)
predict4 = linear_model4.predict(test_x)

print(sqrt(mean_squared_error(predict4, test_y)))

235580.73034317626


In [127]:
# 10번 샘플링 
bagging_model = BaggingRegressor(base_estimator=decision_tree_model, n_estimators=10)
linear_model5 = bagging_model.fit(train_x, train_y)
predict5 = linear_model5.predict(test_x)

print(sqrt(mean_squared_error(predict5, test_y)))

238369.8101331218


In [132]:
# 50번 샘플링 
bagging_model = BaggingRegressor(base_estimator=decision_tree_model, n_estimators=50)
linear_model6 = bagging_model.fit(train_x, train_y)
predict6 = linear_model6.predict(test_x)

print(sqrt(mean_squared_error(predict6, test_y)))

233660.9653609305


In [131]:
# 100번 샘플링 
bagging_model = BaggingRegressor(base_estimator=decision_tree_model, n_estimators=100)
linear_model7 = bagging_model.fit(train_x, train_y)
predict7 = linear_model7.predict(test_x)

print(sqrt(mean_squared_error(predict7, test_y)))

230679.33347375528


In [133]:
# 500번 샘플링 
bagging_model = BaggingRegressor(base_estimator=decision_tree_model, n_estimators=500)
linear_model8 = bagging_model.fit(train_x, train_y)
predict8 = linear_model8.predict(test_x)

print(sqrt(mean_squared_error(predict8, test_y)))

232308.17221605638


In [134]:
"""
=> 모델의 샘플링 수가 증가할수록 예측성능은 좋아지지만, 특정 임계점 초과하는 경우에는 성능저하가 발생함
"""

'\n=> 모델의 샘플링 수가 증가할수록 예측성능은 좋아지지만, 특정 임계점 초과하는 경우에는 성능저하가 발생함\n'